In [3]:
import csv
import string
import re
import keras
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
from nltk import sent_tokenize, word_tokenize, pos_tag, trigrams
from tqdm.notebook import tqdm
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

In [4]:
corpus = []
with open('corpus.csv') as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        corpus.append(line[0])
        

In [5]:
# Lemmatizer and wordlist in case one wants to filter on specific words

lemmatizer = WordNetLemmatizer()

word_list = []
with open('word_file.csv') as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        word_list.append(line[0])
    

In [19]:
#spam_filter = keras.models.load_model('spam_filter')
#word2idx = np.load('word2idx.npy', allow_pickle='TRUE').item()

In [6]:
# Function to convert a treebank PoS tag to a Wordnet PoS tag 
def get_wordnet_pos(treebank_tag):

    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.VERB

In [7]:
def predict(sent_tokens):
    sent_idxs = [word2idx.get(w,1) for w in sent_tokens]
    sent_padded = pad_sequences([sent_idxs], maxlen=15)
    pred = spam_filter.predict([sent_padded])
    return np.argmax(pred)
    
    

In [23]:
# Pick out sentences from the article corpus by checking all conditions
sent_corpus = []

for article in tqdm(set(corpus[:2000]), position=0, leave=True):
    for sentence in sent_tokenize(article):
        
        stripped_sentence = sentence.strip('"')
        
        sent_tokens = word_tokenize(stripped_sentence)
        
        # Complete sentence corpus
        if (4 < len(sent_tokens) <= 15 and 
            '??' not in sentence and 
            '\xa0' not in sentence and
            stripped_sentence[-1] in string.punctuation and
            not re.findall('[A-Z]{5,}', sentence) and
            not stripped_sentence[0].islower()):
        
            # Standard sent corpus w/o additional filtering
            sent_corpus.append([stripped_sentence])    
            
        
            # Further filtered sentence corpus using word lemmas and PoS tags:
        
            #pos_tags = pos_tag(sent_tokens)
            
            #approved = [lemmatizer.lemmatize(sent_tokens[i], pos=get_wordnet_pos(pos_tags[i][1])) in word_list 
             #           or sent_tokens[i] in string.punctuation 
              #          or pos_tags[i][1] == 'NNP'
               #         for i in range(len(sent_tokens))]
            
            #if all(approved):
             #   sent_corpus.append([stripped_sentence])
                


In [1]:
# Same as above but using the spam filter model
#sent_corpus = []

#for article in tqdm(set(corpus[:100]), position=0, leave=True):
 #   for sentence in sent_tokenize(article):
  #      sent_tokens = word_tokenize(sentence)
        
   #     if 4 < len(sent_tokens) <= 15:
    #        if predict(sent_tokens) == 1:
     #           sent_corpus.append(sentence)



In [24]:
len(sent_corpus)

7328

In [17]:
# Save to CSV file
#with open('complete_sent_corpus.csv', 'w', newline='\n') as csvfile:
 #   wr = csv.writer(csvfile)
  #  for line in tqdm(sent_corpus):
   #     wr.writerow(line)
            

# Save trigrams for later use

In [ ]:
# Extract n-grams from corpus to judge possibility of sentences
word_trigrams = {}

for article in tqdm(set(corpus), position=0, leave=True):
    for sentence in sent_tokenize(article):
        sent_tokens = word_tokenize(sentence)
        trigr_list = trigrams(sent_tokens)
        for trigr in trigr_list:
            if trigr in word_trigrams.keys():
                word_trigrams[trigr] += 1
            else:
                word_trigrams[trigr] = 1

In [8]:
len(word_trigrams.keys())

17503375

In [ ]:
reduced_word_trigrams = {}
for key, val in tqdm(word_trigrams.items()):
    if val > 1:
        reduced_word_trigrams.update({key: val})

In [9]:
len(reduced_word_trigrams.keys())

4826155

In [ ]:
# Save trigrams to use later when picking sentences with BERT
# np.save('trigrams.npy', reduced_word_trigrams)